In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import datetime
import os
import sys
import time

In [3]:
sys.path.append("/workspace/main-pe/")

In [4]:
import tensorflow as tf
tf.__version__

'2.0.0-alpha0'

In [5]:
from data.datasets.mpii import MpiiSinglePerson
from data.utils.data_utils import TEST_MODE, TRAIN_MODE, VALID_MODE
from data.loader import BatchLoader

In [6]:
from model import config
from model import callbacks
from model.utils import pose_format

In [7]:
from experiments.common import exp_init

In [8]:
# for custom objects
from model.activations import channel_softmax_2d
from model.losses import reconstruction_loss, pose_loss

# datasets

In [9]:
conf = {
    'pose_blocks': 1,
    'batch_size': 16,
    'dim': 2,
    'n_joints': 16,
    'dataset_name': 'mpii',
    'n_epochs': 2
}

In [10]:
mpii_path = '/workspace/datasets/mpii'
mpii = MpiiSinglePerson(mpii_path, dataconf=config.mpii_dataconf)

In [11]:
data_tr_mpii = BatchLoader(mpii,
                           ['frame'],
                           ['frame'] + ['pose'] * conf['pose_blocks'],
                           TRAIN_MODE,
                           batch_size=conf['batch_size'],
                           shuffle=False)

y_dict ['frame', 'pose'], allkeys ['frame', 'frame', 'pose']


In [12]:
mpii_eval = MpiiSinglePerson(mpii_path, dataconf=config.mpii_dataconf)
data_val_mpii = BatchLoader(
    mpii_eval, 
    ['frame'], 
    ['frame', 'pose', 'afmat', 'headsize'], 
    mode=VALID_MODE,
    shuffle=False)

len(data_val_mpii)

y_dict ['frame', 'pose', 'afmat', 'headsize'], allkeys ['frame', 'frame', 'pose', 'afmat', 'headsize']


124

# 1. non nested model

In [ ]:
from model.networks.old_old_mbm import MultiBranchModel
model = MultiBranchModel(dim=conf['dim'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
conf['exp_type'] = '2d_NN_OLD_MBM_SAVE_TEST_NB'
model_folder = exp_init(conf, base_folder='/workspace')

In [ ]:
from model import callbacks
cb_list = []
cb_list.append(callbacks.SaveModel(model_folder))
model.train(data_tr_mpii, steps_per_epoch=10, model_folder=model_folder, n_epochs=2, cb_list=cb_list)

## loading

In [ ]:
checkpoint = model_folder + "/model_001.h5"

custom_objects = {
    '_channel_softmax_2d': channel_softmax_2d(),
    '_rec_loss': reconstruction_loss(),
    '_pose_loss': pose_loss()
}

In [ ]:
from model.networks.old_old_mbm import MultiBranchModel
print("Loading from %s" % checkpoint)
eval_model = MultiBranchModel(dim=conf['dim'], nb_pose_blocks=conf['pose_blocks'])
eval_model.load(checkpoint, custom_objects=custom_objects)

# 2. Nested

In [ ]:
from model.networks.multi_branch_model import MultiBranchModel
model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

In [ ]:
conf['exp_type'] = '2d_NEW_MBM_SAVE_TEST_NB'
model_folder = exp_init(conf, base_folder='/workspace')

In [ ]:
from model import callbacks
cb_list = []
cb_list.append(callbacks.SaveModel(model_folder))
model.train(data_tr_mpii, steps_per_epoch=10, model_folder=model_folder, n_epochs=2, cb_list=cb_list)

## load nested

In [ ]:
checkpoint = model_folder + "/model_001.h5"

custom_objects = {
    '_channel_softmax_2d': channel_softmax_2d(),
    '_rec_loss': reconstruction_loss(),
    '_pose_loss': pose_loss()
}

In [ ]:
from model.networks.multi_branch_model import MultiBranchModel
print("Loading from %s" % checkpoint)
eval_model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
eval_model.load(checkpoint, custom_objects=custom_objects)

# 3. semi nested

In [15]:
from model.networks.old_mbm import MultiBranchModel
model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
model.build()

BUILDING POSE
BUILDING STEM
pose shape (None, 16, 2), vis shape (None, 16, 1), concat shape (None, 16, 3)
pose model summary
Model: "pose_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
stem1_conv (Conv2D)             (None, 128, 128, 32) 864         input_7[0][0]                    
__________________________________________________________________________________________________
stem1_bn (BatchNormalizationV2) (None, 128, 128, 32) 96          stem1_conv[0][0]                 
__________________________________________________________________________________________________
stem1 (Activation)              (None, 128, 128, 32) 0         

/usr/local/lib/python3.5/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


<class 'tensorflow.python.framework.ops.Tensor'> <class 'tensorflow.python.framework.ops.Tensor'>
Shape z_a HELLO (None, 8, 8, 2048)
Shape z_p (None, 16, 3)
Shape concat (None, 8, 8, 2048)
Outputs shape [(None, 256, 256, 3), (None, 16, 3)]
rec y_pred shape (None, 256, 256, 3)
Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 262, 262, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 64) 9472        conv1_pad[0][0]                  
_____________

In [16]:
conf['exp_type'] = '2d_OLD_1b_MBM_SAVE_TEST_NB'
model_folder = exp_init(conf, base_folder='/workspace')

Conducting experiment for 2 epochs and 1 blocks in folder /workspace/pe_experiments/exp_20190516_1740_2d_OLD_1b_MBM_SAVE_TEST_NB__1b_bs16


In [17]:
from model import callbacks
cb_list = []
cb_list.append(callbacks.SaveModel(model_folder))
model.train(data_tr_mpii, steps_per_epoch=10, model_folder=model_folder, n_epochs=2, cb_list=cb_list)

Training with 1 callbacks
Epoch 1/2
 9/10 [==========================>...] - ETA: 13s - loss: 37.5624 - old_decoder_loss: 34.4386 - pose_model_loss: 3.1239
Trying to save model @epoch=001 to /workspace/pe_experiments/exp_20190516_1740_2d_OLD_1b_MBM_SAVE_TEST_NB__1b_bs16/model_001.h5
10/10 [==============================] - 119s 12s/step - loss: 37.0791 - old_decoder_loss: 33.9908 - pose_model_loss: 3.0883
Epoch 2/2
 9/10 [==========================>...] - ETA: 0s - loss: 33.7660 - old_decoder_loss: 31.4399 - pose_model_loss: 2.3262
Trying to save model @epoch=002 to /workspace/pe_experiments/exp_20190516_1740_2d_OLD_1b_MBM_SAVE_TEST_NB__1b_bs16/model_002.h5
10/10 [==============================] - 9s 869ms/step - loss: 33.4494 - old_decoder_loss: 31.1120 - pose_model_loss: 2.3374


In [18]:
checkpoint = model_folder + "/model_001.h5"
# checkpoint = '/workspace/pe_experiments/exp_20190516_1028_2d_OLD_1b_MBM_SAVE_TEST_NB__1b_bs16/model_001.h5'

custom_objects = {
    '_channel_softmax_2d': channel_softmax_2d(),
    '_rec_loss': reconstruction_loss(),
    '_pose_loss': pose_loss()
}

In [19]:
from model.networks.multi_branch_model import MultiBranchModel
print("Loading from %s" % checkpoint)
eval_model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
eval_model.load(checkpoint, custom_objects=custom_objects)

Loading from /workspace/pe_experiments/exp_20190516_1740_2d_OLD_1b_MBM_SAVE_TEST_NB__1b_bs16/model_001.h5
Processing layer conv1
preprocessing layer conv1 (origin load_weights)
Processing layer bn_conv1
preprocessing layer bn_conv1 (origin load_weights)
Processing layer res2a_branch2a
preprocessing layer res2a_branch2a (origin load_weights)
Processing layer bn2a_branch2a
preprocessing layer bn2a_branch2a (origin load_weights)
Processing layer res2a_branch2b
preprocessing layer res2a_branch2b (origin load_weights)
Processing layer bn2a_branch2b
preprocessing layer bn2a_branch2b (origin load_weights)
Processing layer res2a_branch2c
preprocessing layer res2a_branch2c (origin load_weights)
Processing layer res2a_branch1
preprocessing layer res2a_branch1 (origin load_weights)
Processing layer bn2a_branch2c
preprocessing layer bn2a_branch2c (origin load_weights)
Processing layer bn2a_branch1
preprocessing layer bn2a_branch1 (origin load_weights)
Processing layer res2b_branch2a
preprocessing 

preprocessing layer bn5c_branch2c (origin load_weights)
Processing layer old_decoder
preprocessing layer old_decoder (origin load_weights)
layer is a model named old_decoder

 converting nested model (in layer old_decoder)!
preprocessing layer conv2d_3 (origin convert_nested_model)
preprocessing layer batch_normalization_v2_3 (origin convert_nested_model)
preprocessing layer conv2d_4 (origin convert_nested_model)
preprocessing layer batch_normalization_v2_4 (origin convert_nested_model)
preprocessing layer conv2d_5 (origin convert_nested_model)
preprocessing layer batch_normalization_v2_5 (origin convert_nested_model)
preprocessing layer conv2d_6 (origin convert_nested_model)
preprocessing layer batch_normalization_v2_6 (origin convert_nested_model)
preprocessing layer conv2d_7 (origin convert_nested_model)
preprocessing layer batch_normalization_v2_7 (origin convert_nested_model)
preprocessing layer conv2d_8 (origin convert_nested_model)
preprocessing layer batch_normalization_v2_8 (o

rec y_pred shape (None, 256, 256, 3)


In [ ]:
from model.networks.multi_branch_model import MultiBranchModel
print("Loading from %s" % checkpoint)
eval_model = MultiBranchModel(dim=conf['dim'], n_joints=conf['n_joints'], nb_pose_blocks=conf['pose_blocks'])
eval_model.load(checkpoint, custom_objects=custom_objects)

[autoreload of tensorflow.python.keras.layers.core failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/lib/python3.5/dist-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/usr/lib/python3.5/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/lib/python3.5/importlib/__init__.py", line 147, in reload
    raise ImportError(msg.format(name), name=name)
ImportError: module model.networks.old_pose not in sys.modules
]


Loading from /workspace/pe_experiments/exp_20190516_1740_2d_OLD_1b_MBM_SAVE_TEST_NB__1b_bs16/model_001.h5
Processing layer conv1
preprocessing layer conv1 (origin load_weights)
Processing layer bn_conv1
preprocessing layer bn_conv1 (origin load_weights)
Processing layer res2a_branch2a
preprocessing layer res2a_branch2a (origin load_weights)
Processing layer bn2a_branch2a
preprocessing layer bn2a_branch2a (origin load_weights)
Processing layer res2a_branch2b
preprocessing layer res2a_branch2b (origin load_weights)
Processing layer bn2a_branch2b
preprocessing layer bn2a_branch2b (origin load_weights)
Processing layer res2a_branch2c
preprocessing layer res2a_branch2c (origin load_weights)
Processing layer res2a_branch1
preprocessing layer res2a_branch1 (origin load_weights)
Processing layer bn2a_branch2c
preprocessing layer bn2a_branch2c (origin load_weights)
Processing layer bn2a_branch1
preprocessing layer bn2a_branch1 (origin load_weights)
Processing layer res2b_branch2a
preprocessing 

preprocessing layer old_decoder (origin load_weights)
layer is a model named old_decoder

 converting nested model (in layer old_decoder)!
preprocessing layer conv2d_3 (origin convert_nested_model)
preprocessing layer batch_normalization_v2_3 (origin convert_nested_model)
preprocessing layer conv2d_4 (origin convert_nested_model)
preprocessing layer batch_normalization_v2_4 (origin convert_nested_model)
preprocessing layer conv2d_5 (origin convert_nested_model)
preprocessing layer batch_normalization_v2_5 (origin convert_nested_model)
preprocessing layer conv2d_6 (origin convert_nested_model)
preprocessing layer batch_normalization_v2_6 (origin convert_nested_model)
preprocessing layer conv2d_7 (origin convert_nested_model)
preprocessing layer batch_normalization_v2_7 (origin convert_nested_model)
preprocessing layer conv2d_8 (origin convert_nested_model)
preprocessing layer batch_normalization_v2_8 (origin convert_nested_model)
preprocessing layer conv2d_9 (origin convert_nested_model

rec y_pred shape (None, 256, 256, 3)


## eval

In [ ]:
data = data_val_mpii.get_data(1, VALID_MODE)
print(data['frame'].shape)
pred = eval_model.predict(data['frame'])

In [ ]:
import matplotlib.pyplot as plt

n_cols = 2
n_rows = len(data['frame'])
fig = plt.figure(figsize=(8, 75))

print(len(data['frame']))
print(len(pred[0]))

i_img = 1
for i, data_img in enumerate(data['frame']):
    fig.add_subplot(n_rows, n_cols, i_img)
    plt.imshow(data_img)
    
    pred_img = pred[0][i]
    fig.add_subplot(n_rows, n_cols, i_img + 1)
    plt.imshow(pred_img)
    
    i_img += 2
    
plt.show()

In [ ]:
import h5py
import numpy as np
hf = h5py.File(checkpoint, 'r')

In [ ]:
hf.keys()

In [ ]:
a = hf['model_weights']

In [ ]:
print(type(a['pose_model']['stem1_conv']))
b = a['pose_model']['stem1_conv']
list(b.items())

In [ ]:
n1 = a['pose_model']['stem1_conv']
n1 = np.array(n1)
n1.shape

In [ ]:
a['pose_model'].keys()
